In [ ]:
import geopandas as gpd
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
import numpy as np
import rasterio
import math

### Refinement of Agricultural Plot Data

In [ ]:
# Load plot- and mowing classification files:

mow_class = rxr.open_rasterio("mow_class") # a list of binary mowing classification files for the lower Engadine for each Sentinel-2 overflight in Summer 2018
mow_class_dates = os.listdir("mow_class") # retrieves the names of the respective files

agri_plots = gpd.read_file("agri_plot_engadine.gpkg") # contains the agricultural plot data of the Canton of Grisons

In [ ]:
# checking each plot whether for the median value of the binary mowing classification within the plot

for i in range(len(mow_class)):   # itterating over the list containig the mowing classification for each date     
        bin_list_per_area = []    # providing an empty list to store the median value of each plot 
    
        for j in range(len(agri_plots)):        
            if agri_plots.iloc[j:j+1, :] is None: # check if data within plot
                continue
            else:
                clipped = mow_class[i].rio.clip(agri_plots.iloc[j:j+1, :].geometry, NF.crs, all_touched=True ) # clip the mowing classification file to the extent of the plot
                m = clipped.median(skipna=True).values # claculate the median value
        
            bin_list_per_area.append(int(m)) # appending the median value to the list
    
        agri_plots[f"{dates[i]}"] = bin_list_per_area # saving the list with the median values as new collumn in the plot dataset.

In [ ]:
# get first mowing for each plot (first time value = 1 at each plot)
dates = ['2018-06-01','2018-06-03','2018-06-06','2018-06-08','2018-06-11','2018-06-13','2018-06-16','2018-06-18','2018-06-21','2018-06-23','2018-06-26','2018-06-28','2018-07-01','2018-07-03','2018-07-06','2018-07-08','2018-07-11','2018-07-13','2018-07-16','2018-07-18','2018-07-21','2018-07-23','2018-07-26','2018-07-28','2018-07-31']
# list of possible dates

agri_plots["mowed_first"] = "" # providing empty collumn to store the date of the first mowing

for index, row_tuple in agri_plots.iterrows():   # iterate over the plots and find the respective first date of mowing
    for date in dates:
        if row_tuple[date] == 1:
            agri_plots.loc[index:index+1, "mowed_first"] = date

In [ ]:
# create the merging criterion by combining the date of the first mowing and the management type
agri_plots["merge_crit"] = agri_plots["nutzung_de"] + ": " + agri_plots["mowed_first"]

In [ ]:
# merge areas when they touch each other and the merging criterion is equal
merge_crit_list = agri_plots["merge_crit"].values.tolist() # retrieve all "merge_crit"-values from list
merge_crit_list = list( dict.fromkeys(merge_crit_list)) # delete duplicates from the list

for i in range(len(merge_crit_list)):    
    cat_polys = agri_plots.loc[agri_plots['merge_crit'] == merge_crit_list[i]] # select all polys with same "merge_crit"-value
    cat_merged = cat_polys.dissolve().explode() # unionize the ones that touch each other
    
    for index, poly in cat_merged.iterrows(): # save to new data frame
        new_df.append({'nutzung_de': poly['nutzung_de'], 'mowed_first': poly['mowed_first'], 'merge_crit': poly['merge_crit'], 'geometry': poly['geometry']})

    agri_plots_merged = gpd.GeoDataFrame(new_df)

### Calculation of the Spectral Metrics (on AVIRIS-NG PCs)

In [ ]:
ang_PC_data = rxr.open_rasterio('ang_PCs.tiff') # load AVIRIS-NG dataset containing the PCs

In [ ]:
# preparing a function which allows to calculate the spectral diversity metrics for AVIRIS-NG PCs
crs = 'epsg:2056'

def aviris_var_cv(ang_data, agri_plots):
    column_name_var = 'VAR'
    column_name_CV = 'CV'
      
    for i in range(len(agri_plots)): # iterating over the refined plot dataset
        list_var = [] # preparing lists to store the spectral diversity values. This allows to calculate an average value when all layers are analyzed.
        list_cv = []
        index = NF.iloc[i:i+1, :].index
                
        for j in range(len(ang_data)):    
            try: 
                clipped_raster = ang_data[j].rio.clip(agri_plots.iloc[i:i+1, :].geometry, crs=crs, invert=False) # clip PC-raster dataset with a single plot
                    
            except rxr.exceptions.NoDataInBounds: # exception needed to be built in because an error ocurred when the clipped raster was empty
                continue                    

            var = clipped_raster.var() # calculating the variance within a plot of one PC
            variance_val = var.values
            list_var.append(variance_val) 
            
            std = clipped_raster.std() # calculating the CV within a plot of one PC
            avg = clipped_raster.mean()
            cv = std / avg
            list_cv.append(cv.values)
            
        agri_plots.loc[index, column_name_var] = round(np.mean(list_var), 10) # calculating the average variance value for each respective plot and saving in the dataframe
        agri_plots.loc[index, column_name_CV] = round(np.mean(list_cv), 10) # calculating the average CV value for each respective plot and saving in the dataframe

    return agri_plots

In [ ]:
# Calculating the diversity metrics for the three first PCs
ang_spectral_diversity_3PCs = aviris_var_cv(ang_PC_data[:3], agri_plots_merged)

In [1]:
# Calculating the diversity metrics for the NDVI of AVIRIS-NG
ang_NDVI_bands = rxr.open_rasterio("ang_NDVI_bands.tiff")

b57 = ang_NDVI_bands[0]
b91 = ang_NDVI_bands[1]
ndvi = (b91 - b57) / (b91 + b57) # Calculate NDVI

ang_spectral_diversity_NDVI = aviris_var_cv(ndvi, agri_plots_merged)

NameError: name 'rxr' is not defined

#### Spectral Diversity Calculations for SwissImage RS and Sentinel-2
The spectral diversity of the Sentinel-2 products is calculated in the same way. For SwissImage RS, the calculation must be split between the SwissImage RS image tiles, but then it is the same again.

Many other programming steps (data preparation, refinement, analysis and visualization) were required for this work. However, these two steps of data analysis were the core of the work and are therefore presented here.